In [1]:
def remove_suffix(title):
    suffices=[": novellroman",": en roman",": en berättelse",": detektivroman",": gymnasial berättelse",": en berättelse",": Roman",": dramatisk roman",": kriminalroman",": självbiografisk berättelse",": historisk roman",": historisk äventyrsroman",": en kärleksthriller",": en äventyrsberättelse",": en ungdomsbok",": [roman]" ]
    for x in suffices:
        if x in title:
            title=title.replace(x,"")
            break
    return title

In [2]:
from kblab import Archive
import requests
from requests.auth import HTTPBasicAuth
from lxml import etree
import xml.etree.ElementTree as ET
import time
romaner={"meta.host_title": "Welfare state analytics"}
blm={"label": "BONNIERS", "tags": "issue"}
afton={"tags": "issue"}

def titles_mapping(filter=romaner,max_number=6000):
    with open('/home/liamtabibzadeh/Documents/API_credentials.txt', 'r') as file:
        pw = file.read().replace('\n', '')
    a = Archive("https://datalab.kb.se", auth=("demo", pw))
    
    books_meta={}
    for package_id in a.search(filter, max=max_number):
        for x in a.get(package_id):
            if "aip.mets.metadata" in x:
                for i in range(5):
                    backoff_time = 0.1 * (2 ** i)
                    meta=requests.get(f"https://datalab.kb.se/{package_id}/{x}", auth=HTTPBasicAuth("demo", pw),stream=True)
                    if meta.status_code == 200:
                        root=ET.fromstring(meta.text)
                        meta_data=root.attrib
                        title=meta_data["LABEL"]
                        title=remove_suffix(title)
                        books_meta[title]=package_id
                
                        break
            
                    else:
                        time.sleep(backoff_time)
                break
    return books_meta

In [3]:
dictt=titles_mapping()

In [4]:
url_excel="/home/liamtabibzadeh/Documents/kb info/meta.xlsx"
import pandas as pd
meta_data=pd.read_excel(url_excel)
meta_data["Datalab ID"]=""
meta_data

,Titel,Efternamn,Förnamn,Utgivningsår,Libris-ID,Regina-ID,Datalab ID
0,Dödskniven,Carlberg,Berndt,1955,9sl724zm2k5tcjg,000471915,
1,Mord i eldsken?,Carlberg,Berndt,1953,3ld1vxrf38s2spq,000471909,
2,Mord i alla fall,Blomberg,Stig O.,1954,6ph4z1vj5vbwxsc,000471912,
3,Den gamla fabeln om Tälje tokar.,Kassius,Torsten,1950,p60mfhw12c02hbg,001855170,
4,En boxare försvinner,Ahlqvist,Edwin,1951,tb442pk55tj574z,002463477,
...,...,...,...,...,...,...,...
2778,Allt ändrade sig,Petersson,Linné,1965,5ng3zsph2n2krld,000487672,
2779,Erotikon,Widerberg,Bo,1965,r82pt0v3483b1rc,000550724,
2780,Lurendrejaren i Rye,Månsson,Nils E,1965,5ng3g7dh2cww311,000594810,
2781,Årstider,Nyström,Lars,1965,n5zlfbw03hx4r4z,000468850,


In [5]:
meta_data.iloc[1]["Titel"]

'Mord i eldsken? '